# 1、使用FUNCTION_CALL模式

In [3]:

# 1、导入相关包
import os
import dotenv
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import Tool

# 2、配置环境变量
dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 3、创建大模型实例
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0
)

# 4、创建搜索实例
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
search = TavilySearchResults(
    max_results=3
)

# 5、创建搜索工具
search_tool = Tool(
    name='Search',
    func=search.run,
    description='搜索互联网上的信息'
)

# 6、创建提示词模板（以ChatPromptTemplate为例）
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个乐于助人的ai助手, 根据用户的提问, 必要时调用Search工具, 使用互联网检索数据'),
        ('system', '{agent_scratchpad}'),
        ('system', '{chat_history}'),  # 新增chat_history变量, 用于记录上下文的记忆
        ('human', '{input}'),
    ]
)

# 7、提供记忆的实例：ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history',  # 此值需要与提示词模板中的记录记忆的{chat_history}同名
)

# 8、获取Agent实例：create_tool_calling_agent()
agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt_template,
)

# 9、获取AgentExecutor的实例
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    memory=memory,
)

# 10、通过AgentExecutor的实例调用invoke(), 获取响应结果
response = agent_executor.invoke({'input': '查询今天2025年11月16日北京的天气情况'})

# 11、处理响应结果
print(response)



> Entering new AgentExecutor chain...

Invoking: `Search` with `2025年11月16日 北京 天气预报`


[{'title': '北京市在十一月2025 的天氣：氣溫與氣候', 'url': 'https://zh.climate-data.org/%E4%BA%9E%E6%B4%B2/%E4%B8%AD%E5%9B%BD/%E5%8C%97%E4%BA%AC%E5%B8%82/%E5%8C%97%E4%BA%AC%E5%B8%82-134/t/%E5%8D%81%E4%B8%80%E6%9C%88-11/', 'content': '##\n\n# 北京市 在 十一月 的天氣情況\n\n您打算在 十一月 2025 到 北京市 度假，希望天氣宜人嗎？在這裡您可以找到 北京市 在 十一月 的所有天氣資訊：\n\n 北京市 在 十一月 的天氣與氣候概況\n 北京市 每月天氣 // 天氣平均值\n 北京市 在 十一月 的天氣平均狀況\n 北京市 的當前溫度及天氣預報\n\n## 北京市 在 十一月 的天氣\n\n|  |  |  |  |  |  |\n ---  ---  --- |\n|  | 溫度 十一月 | 3.8°C |  | 降水量 十一月 | 15mm |\n|  | 溫度 十一月 最大值 | 10.1°C |\n|  | 溫度 十一月 分鐘 | -1.4°C |\n\n|  |  |  |\n --- \n|  | 溫度 十一月 | 3.8°C |\n|  | 溫度 十一月 最大值 | 10.1°C |\n|  | 溫度 十一月 分鐘 | -1.4°C |\n|  | 降水量 十一月 | 15mm |\n\n## 十一月 北京市 的气候 [...] | 9. 十一月 | 6 °C | 12 °C | 0 °C | 4.9 mm | 8.5 |\n| 10. 十一月 | 6 °C | 12 °C | 1 °C | 2.4 mm | 7.8 |\n| 11. 十一月 | 6 °C | 12 °C | 0 °C | 6.3 mm | 8.1 |\n| 12. 十一月 | 5 °C | 12 °C | -0 °C | 3.3 mm | 8.2 |\n| 13. 十一月 | 5 °C | 11 

In [4]:
# 10、通过AgentExecutor的实例调用invoke(), 获取响应结果
response = agent_executor.invoke({'input': '上海的呢'})

# 11、处理响应结果
print(response)



> Entering new AgentExecutor chain...

Invoking: `Search` with `2025年11月16日上海的天气情况`


[{'title': '上海市在十一月2025 的天氣：氣溫與氣候 - Climate-Data.org', 'url': 'https://zh.climate-data.org/%E4%BA%9E%E6%B4%B2/%E4%B8%AD%E5%9B%BD/%E4%B8%8A%E6%B5%B7%E5%B8%82-890/r/%E5%8D%81%E4%B8%80%E6%9C%88-11/', 'content': '##\n\n# 上海市 在 十一月 的天氣情況\n\n您打算在 十一月 2025 到 上海市 度假，希望天氣宜人嗎？在這裡您可以找到 上海市 在 十一月 的所有天氣資訊：\n\n >>上海市 在 十一月 的天氣與氣候概況\n >>上海市 每月天氣 // 天氣平均值\n >>上海市 在 十一月 的天氣平均狀況\n >>上海市 的當前溫度及天氣預報\n\n在 十一月，上海市 在 06.11 記錄了當天最高溫，達到 21.5°C，而最低溫則出現在 30.11，為 6.7°C。\n月初三分之一期間，平均最高溫為 20°C，最低溫為 12.1°C。\n第二個三分之一期間最高溫相似，約為 17.3°C，最低溫則接近 10.4°C。\n最後幾天的最高溫為 15.3°C，最低溫為 10.6°C。\n最熱的連續10天期間為 1-10，平均最高溫達 15.7°C，\n而最冷的連續10天則落在 21-10，最低溫約為 12.7°C。 [...] |  | 溫度 (°C) | 溫度 最大值 (°C) | 溫度 分鐘 (°C) | 降水量 (mm) | 平均日照時數 (小時) |\n ---  ---  --- |\n| 1. 十一月 | 16 °C | 20 °C | 12 °C | 3.9 mm | 0.0 |\n| 2. 十一月 | 15 °C | 19 °C | 11 °C | 5.6 mm | 0.0 |\n| 3. 十一月 | 15 °C | 20 °C | 11 °C | 3.1 mm | 0.0 |\n| 4. 十一月 | 16 °C | 20 °C | 13 °C | 2.6 mm | 0.

# 2、使用ReAct模式


举例1：使用网络上现成的提示词模板

In [5]:
from langchain import hub
# 1、导入相关的包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

# 加载环境变量
dotenv.load_dotenv()

# 2、定义搜索化工具
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

tools = [search]

# 3、使用LangChain Hub中的官方ReAct提示模板（从官方获取 https://smith.langchain.com/hub/hwchase17/react-chat ）
prompt_template = hub.pull('hwchase17/react-chat')

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history',
)

# 4、定义LLM
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 5、创建Agent对象
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template
)

# 6、创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

# 7、测试
response = agent_executor.invoke({'input': '今天2025年11月16日北京的天气怎么样？'})
print(response)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: 2025年11月16日 北京 天气
```[{'title': '北京市北京市气象台发布大风蓝色预警信号', 'url': 'http://www.nmc.cn/publish/alarm/11000041600000_20251116184246.html', 'content': '市气象台2025年11月16日18时40分降级发布大风蓝色预警信号：目前我市风力有所减弱，预计16日夜间至17日白天，我市大部分地区有3、4级偏北风，阵风6、7级，山区阵风可达8级以上，请注意防范。\n\n防御指南：\n\n1.政府及相关部门按照职责做好防大风工作；   \n2.关好门窗，加固围板、棚架、广告牌等易被风吹动的搭建物，妥善安置易受大风影响的室外物品，遮盖建筑物资；   \n3. 相关水域水上作业和过往船舶采取积极的应对措施，如回港避风或者绕道航行等；   \n 4.行人注意尽量少骑自行车，刮风时不要在广告牌、临时搭建物等下面逗留；   \n5.有关部门和单位注意森林、草原等防火。\n\n推荐服务\n\n关于我们联系方式网站声明业务信息网站动态网站地图触摸屏版\n\n国家气象中心 版权所有 Copyright©2009-2025\n\n制作维护：国家气象中心预报系统开放实验室 地址：北京市中关村南大街46号 邮编：100081\n\n京公网安备 11040102700100号    京ICP备05055842号\n\n中央气象台   \n科技创新服务 [...] + CMA全球天气模式\n  + CMA全球集合模式\n  + CMA区域模式\n  + CMA区域集合模式\n  + CMA台风模式\n  + 海浪模式\n\n#### 天气预报\n\n天气公报\n\n每日天气提示\n\n春运气象服务专报\n\n气象灾害预警\n\n重要天气提示\n\n重要天气盘点\n\n降水量预报\n\n冻雨预报\n\n气温预报\n\n大风预报\n\n强对流天气预报\n\n中期天气\n\n全球天气预报\n\n全球灾害性天气监测月报\n\

In [6]:
response = agent_executor.invoke({'input': '上海的呢'})

print(response)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: 2025年11月16日上海的天气
```[{'title': '上海市2025年11月16日天气多地降雨最高气温22', 'url': 'https://www.zgjia.com/report/shanghai20251116.html', 'content': '首页\n 旅游天气\n 国际天气\n 空气质量\n 热门地点\n 历史天气\n 天气报告\n 气温排行\n\n首页 天气报告 上海天气报告   上海市2025年11月16日天气多地降雨最高气温22℃ \n\n上海市2025年11月16日（星期日）农历乙巳蛇年九月廿七，天气数据总结：白天最高气温平均温度为22.1度，夜间最低气温平均气温为12.1度；其中，最高气温发生在上海的上海市，温度为22摄氏度，最低气温发生在上海的上海市，温度为22摄氏度；上海大部区域晴天占比为0%，多云天气占比为10.5%，阴天占比为0%，雨天占比为78.9%，雪天占比为0%，天气多地降雨，气温舒适，适宜穿着单衣类服装，不适合户外活动。本站所有气温数据来源于中央气象台。\n\n上海市主要城区天气讯息：\n\n长宁天气\u3000多云转小雨 南风转北风一二级转三至四级\u3000\u300022~14°C\n\n上海天气\u3000多云转小雨 南风转北风一二级转三至四级\u3000\u300022~14°C\n\n宝山天气\u3000多云转小雨 东风转北风一二级转四至五级\u3000\u300021~8°C\n\n崇明天气\u3000晴转小雨 东北风转北风一二级转四至五级\u3000\u300021~7°C\n\n奉贤天气\u3000阴 东南风转北风一二级转三至四级\u3000\u300022~10°C [...] 上海市2025年11月16日天气多地降雨最高气温22℃\n 2025年11月15日周六上海市天气情况汇总\n 上海市2025/11/14星期五天气以晴为主最高气温19度\n 上海市2025年11月13日周四天气以晴为主最高温20℃\n 上海市2025年11月12日

举例2：自定义提示词模板

In [7]:
from langchain import hub
# 1、导入相关的包
import os
import dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent

# 加载环境变量
dotenv.load_dotenv()

# 2、定义搜索化工具
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

search = TavilySearchResults(max_results=1)

tools = [search]

# 3、自定义提示词模板（从官方获取 https://smith.langchain.com/hub/hwchase17/react-chat ）
template = """
Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
"""

prompt_template = PromptTemplate.from_template(template)

# 使用LangChain Hub中的官方ReAct提示模板
# prompt_template = hub.pull('hwchase17/react-chat')

# 提供记忆的实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history',
)

# 4、定义LLM
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 5、创建Agent对象
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template
)

# 6、创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

# 7、测试
response = agent_executor.invoke({'input': '今天2025年11月16日北京的天气怎么样？'})
print(response)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: 2025年11月16日 北京 天气
```[{'title': '北京市北京市气象台发布大风蓝色预警信号', 'url': 'http://www.nmc.cn/publish/alarm/11000041600000_20251116184246.html', 'content': '市气象台2025年11月16日18时40分降级发布大风蓝色预警信号：目前我市风力有所减弱，预计16日夜间至17日白天，我市大部分地区有3、4级偏北风，阵风6、7级，山区阵风可达8级以上，请注意防范。\n\n防御指南：\n\n1.政府及相关部门按照职责做好防大风工作；   \n2.关好门窗，加固围板、棚架、广告牌等易被风吹动的搭建物，妥善安置易受大风影响的室外物品，遮盖建筑物资；   \n3. 相关水域水上作业和过往船舶采取积极的应对措施，如回港避风或者绕道航行等；   \n 4.行人注意尽量少骑自行车，刮风时不要在广告牌、临时搭建物等下面逗留；   \n5.有关部门和单位注意森林、草原等防火。\n\n推荐服务\n\n关于我们联系方式网站声明业务信息网站动态网站地图触摸屏版\n\n国家气象中心 版权所有 Copyright©2009-2025\n\n制作维护：国家气象中心预报系统开放实验室 地址：北京市中关村南大街46号 邮编：100081\n\n京公网安备 11040102700100号    京ICP备05055842号\n\n中央气象台   \n科技创新服务 [...] + CMA全球天气模式\n  + CMA全球集合模式\n  + CMA区域模式\n  + CMA区域集合模式\n  + CMA台风模式\n  + 海浪模式\n\n#### 天气预报\n\n天气公报\n\n每日天气提示\n\n春运气象服务专报\n\n气象灾害预警\n\n重要天气提示\n\n重要天气盘点\n\n降水量预报\n\n冻雨预报\n\n气温预报\n\n大风预报\n\n强对流天气预报\n\n中期天气\n\n全球天气预报\n\n全球灾害性天气监测月报\n\

In [8]:
response = agent_executor.invoke({'input': '上海的呢'})
print(response)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: 2025年11月16日上海的天气
```[{'title': '上海市2025年11月16日天气多地降雨最高气温22', 'url': 'https://www.zgjia.com/report/shanghai20251116.html', 'content': '首页\n 旅游天气\n 国际天气\n 空气质量\n 热门地点\n 历史天气\n 天气报告\n 气温排行\n\n首页 天气报告 上海天气报告   上海市2025年11月16日天气多地降雨最高气温22℃ \n\n上海市2025年11月16日（星期日）农历乙巳蛇年九月廿七，天气数据总结：白天最高气温平均温度为22.1度，夜间最低气温平均气温为12.1度；其中，最高气温发生在上海的上海市，温度为22摄氏度，最低气温发生在上海的上海市，温度为22摄氏度；上海大部区域晴天占比为0%，多云天气占比为10.5%，阴天占比为0%，雨天占比为78.9%，雪天占比为0%，天气多地降雨，气温舒适，适宜穿着单衣类服装，不适合户外活动。本站所有气温数据来源于中央气象台。\n\n上海市主要城区天气讯息：\n\n长宁天气\u3000多云转小雨 南风转北风一二级转三至四级\u3000\u300022~14°C\n\n上海天气\u3000多云转小雨 南风转北风一二级转三至四级\u3000\u300022~14°C\n\n宝山天气\u3000多云转小雨 东风转北风一二级转四至五级\u3000\u300021~8°C\n\n崇明天气\u3000晴转小雨 东北风转北风一二级转四至五级\u3000\u300021~7°C\n\n奉贤天气\u3000阴 东南风转北风一二级转三至四级\u3000\u300022~10°C [...] 上海市2025年11月16日天气多地降雨最高气温22℃\n 2025年11月15日周六上海市天气情况汇总\n 上海市2025/11/14星期五天气以晴为主最高气温19度\n 上海市2025年11月13日周四天气以晴为主最高温20℃\n 上海市2025年11月12日